In [1]:
s = "def greeting(name: str) -> str: \
    return 'Hello ' + name "

In [2]:
import astor,ast

In [3]:
a = ast.parse(s)

In [4]:
#!pipenv install typed-ast

In [5]:
from typed_ast import ast3

In [6]:
a = ast3.parse(s)

In [7]:
a

In [8]:
hypothesis_mappings = {"str": {"decorator":"@given(text())","hypothesis_test":"s"}}

def make_test(a):
    for i in a.body:
        name = i.name
        r = i.returns.id
        args = [ x.arg for x in i.args.args]
        args = ", ".join(args)
        hypothesis_test = hypothesis_mappings[r]["hypothesis_test"]
        t = hypothesis_mappings[r]["decorator"]+"\n"
        t += f'def test_{name}({hypothesis_test}):\n'
        t +=f'    assert type({name}({hypothesis_test})) is {r}'
        return t

In [9]:
print(make_test(a))

@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str


In [10]:
#!pipenv install hypothesis

In [11]:
from hypothesis import given
from hypothesis.strategies import text,builds,lists

def greeting(name: str) -> str: 
    return 'Hello ' + name 
@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str

In [12]:
test_greeting()

In [13]:
from collections import namedtuple

In [14]:
from typing import List
class DataPoint:
    def __init__(self,x:int,y:int):
        self.x = x
        self.y =y

class DataSet:
    def __init__(self,label:str,data:List[DataPoint],backgroundColor:str):
        self.label=label
        self.data=data
        self.backgroundColor = backgroundColor

class Data:
    def __init__(self,datasets:List[DataSet]=None):
        self.datasets=datasets

In [15]:
data1 = Data([DataSet(label="label1",data=[DataPoint(1,2),DataPoint(3,4)],backgroundColor="00b8ff")])

In [16]:
from crawto.Charts.charts import tsne_plot,make_html

In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
from crawto.Charts.charts import tsne_plot,make_html
from hypothesis import given
from hypothesis.strategies import text,builds,lists,floats
#from crawto.Charts.chart_types import 

@given(builds(Data,lists(builds(DataSet,text(),lists(builds(DataPoint,floats(),floats())),text()))))
def test_tsne_chart(d):
    assert type(make_html(d)) is str

In [19]:
test_tsne_chart()

In [20]:
s=open("crawto/Charts/charts.py").read()
a = ast3.parse(s)

In [21]:
a.body

In [22]:
import typed_ast
type(a.body[0]) is typed_ast._ast3.ImportFrom

True

In [23]:
z =a.body[7]

In [24]:
z.args.args[0].annotation.id

'Data'

In [52]:
dir(a.body[4])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'col_offset',
 'level',
 'lineno',
 'module',
 'names']

In [54]:
a.body[4].names[0].name,a.body[4].module,a.body[4].level

('Data', 'Charts.chart_type', 2)

In [26]:
import typing
ast_import_union = typing.Union[typed_ast._ast3.ImportFrom,typed_ast._ast3.Import]

In [70]:
for i in a.body[4].names:
    print(i.name)

Data


In [72]:
name_module_dict = {}
def map_realitive_import_to_object(ast_import):
    if type(ast_import) is typed_ast._ast3.ImportFrom:
        if ast_import.level>0:
            module=ast_import.module
            for i in ast_import.names:
                name_module_dict[i.name] = module.replace(".","/")
    elif type(ast_import) is typed_ast._ast3.Import:
        print("success")
    else:
        print("Fail")
        
for i in a.body:
    map_import_to_object(i)
name_module_dict

success
success
Fail
Fail


{'Data': 'Charts/chart_type'}

In [81]:
b=open("crawto/Charts/chart_type.py").read()
b = ast3.parse(b)

In [86]:
for i in b.body:
    try:
        print(i.name)
    except:
        pass

DataPoint
DataSet
Data


In [94]:
dir(b.body[3])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'bases',
 'body',
 'col_offset',
 'decorator_list',
 'keywords',
 'lineno',
 'name']

In [104]:
b.body[3]

In [163]:
def parse_class_function(ast3_ClassDef):
    for i in ast3_ClassDef.body:
        if i.name == '__init__':
            args = [a.value.id for a in i.body]
            annotations = 

['datasets']


In [115]:
b.body[3].body[0].body[0].value.id

'datasets'

In [155]:
#b.body[3].body[0].args.args[1].arg,
b.body[3].body[0].args.args[1].annotation.value.id

'List'

In [166]:
b.body[3].body[0].args.args[1].annotation

In [162]:
b.body[3].body[0].args.args[1].annotation.slice.value.id

'DataSet'